###  <a href='https://ko-fi.com/P5P1QO6VB' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://storage.ko-fi.com/cdn/kofi4.png?v=3' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a>

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="https://i.ibb.co/w6ngBsW/20231031152934.jpg" alt="20231031152934" width="300">
    <span style="margin: 0 20px;">|</span>
    <img src="https://i.ibb.co/SrxYKmz/20231031152928.jpg" alt="20231031152928" width="400">
</div>

## 原神启动！

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/Invoke/IAekovnI_colab.ipynb)
[![Visitors](https://api.visitorbadge.io/api/visitors?path=https%3A%2F%2Fcolab.research.google.com%2Fgithub%2FVan-wise%2Fsd-colab%2Fblob%2Fmain%2FInvoke%2FIAekovnI_colab.ipynb&label=Page%20View%3A&labelColor=%232ccce4&countColor=%23555555&style=flat)](https://visitorbadge.io/status?path=https%3A%2F%2Fcolab.research.google.com%2Fgithub%2FVan-wise%2Fsd-colab%2Fblob%2Fmain%2FInvoke%2FIAekovnI_colab.ipynb)

---


<font color="red">**更新地址**：https://github.com/Van-wise/sd-colab

**如果遇到报错,请联系up：**
[bilibili](https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244) or [issues](https://github.com/Van-wise/sd-colab/issues)



In [ ]:
# @title (🔆)安装环境

import os
import re
import time
from IPython.utils import capture
from IPython.display import clear_output, display, HTML

# 忽略警告
import warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
!git config --global advice.detachedHead false
!sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' {warnings.__file__}
warnings.filterwarnings("ignore")

# @markdown ---
#@markdown ###<font color="#11659a"> 选择分支：</font>
branchname = "v3.3.0post3" # @param ["v3.3.0post3", "v3.4.0rc2", "main"]
#@markdown #####<font color="#2c9678">☑链接谷歌云盘；☐不使用谷歌云盘。(使用云端模型、备份数据等.)
_use_gdrive = False # @param {type:"boolean"}
#@markdown #####<font color="#2c9678">☑检查是否使用了GPU；☐不检查。
_check_gpu = True  #@param {type:"boolean"}

def check_gpu():
    import tensorflow as tf
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("\033[96m GPU is available.\033[0m", flush=True)
    else:
        print("\033[91m 没有使用GPU,请在代码执行程序-更改运行时类型-设置为GPU!\033[0m", flush=True)
        display(HTML("<img src='https://i.ibb.co/xfb7pB7/check-gpu.png' width='560px'/>"))
        from google.colab import runtime
        runtime.unassign()        

airoot_dir = os.environ['AIROOT_DIR'] = os.getcwd() if 'AIROOT_DIR' not in os.environ else os.environ['AIROOT_DIR']
os.chdir(airoot_dir)  #/content

if _check_gpu:
  check_gpu()

if _use_gdrive and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')
  print('Google Drive 挂载成功！')

start_time = time.time()
!git clone -q --depth 1 -b {branchname} https://github.com/invoke-ai/InvokeAI/ IAekovnI
print('项目克隆成功！')

# @markdown ---
os.chdir('IAekovnI')
main_dir = os.environ['MAIN_DIR'] = os.getcwd() if 'MAIN_DIR' not in os.environ else os.environ['MAIN_DIR']
db_dir = f'{main_dir}/db'

#安装环境
print("正在安装环境,大约需要六分钟...")
with capture.capture_output() as cap:
  !apt-get -qq -y update
  !apt-get -qq install install build-essential python3-opencv libopencv-dev
  !apt-get -qq install -y aria2 git #triton==2.0.0
  !pip install -q xformers==0.0.20 pyngrok pycloudflared -U
  !pip install -q -e .  #用于构建Invoke
  !invokeai-configure --root {db_dir} --yes --skip-sd-weights

!wget -q https://huggingface.co/Vanwise/sd-colab/resolve/main/libtcmalloc_minimal.so.4
os.environ['LD_PRELOAD'] = f'{main_dir}/libtcmalloc_minimal.so.4'
!pip cache purge
end_time = time.time()
print("环境安装完成，耗时：{:.3f}分钟.".format((end_time - start_time) / 60))

In [ ]:
# @title (🔆)MOD！
# @markdown #### 💠用于下载主模型、Lora、Vae、Embedding.
import pandas as pd
import requests
from io import BytesIO
from google.colab import files, drive
from concurrent import futures

# @markdown #### 启用MOD:
enable_mod = True #@param {type:"boolean"}
# @markdown #### 模型下载位置:
modown_dir = "colab" # @param ["colab", "drive"]
if modown_dir == "drive":
  modelr_dir = "/content/drive/MyDrive/sd_models"
else:
  modelr_dir = f'{db_dir}/models'
# @markdown ---
# @markdown ##### [MOD链接](https://docs.google.com/spreadsheets/d/1sp_o8fUBZk8BrThY9w1LeLd2aTuq41sT/edit#gid=149213849):
mod_link = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSbZi5KIL4pOu7vQ-i5dgnJ5NVy8RlqI4y80ViXQU4ILtnR_nkBMHNr6J4UR7AIHA/pub?output=xlsx"# @param {type:"string"}
# @markdown ##### MOD路径:
mod_path = "/content/drive/MyDrive/wise.xlsx"# @param {type:"string"}
# @markdown ##### 代码逻辑:优先使用MOD链接,链接为空使用MOD路径,路径为空使用上传MOD。

def checkmod_link(link):
  response = requests.get(link)
  if response.status_code == 200:
      content = response.content
      try:
          df = pd.read_excel(BytesIO(content))
          print("MOD读取成功!")
          return df
          #print(df.head())
      except Exception as e:
          print("无法正确读取文件,请检查文件格式是否为'.xlsx' :", str(e))
  else:
      print("无法获取文件,请检查链接是否正确!")

def checkmod_path(path):
  try:
    df = pd.read_excel(path)
    print("MOD读取成功!")
    return df
    #print(df.head())
  except Exception as e:
    print("无法正确读取文件,请检查文件格式是否为'.xlsx' :", str(e))

def checkmod_up():
    while True:
        try:
            uploaded = files.upload()
            if not uploaded:
                print("取消上传,使用默认MOD!")
                mod = 'https://github.com/Van-wise/sd-colab/raw/main/wise.xlsx'
                df = pd.read_excel(mod)
                break
            mod_file = os.path.join(os.getcwd(), list(uploaded.keys())[0])
            df = pd.read_excel(mod_file)
            print("MOD读取成功!")
            break  # Exit the loop if the file is successfully read

        except (ValueError, Exception) as e:
            os.remove(mod_file)
            print("无法正确读取文件，请检查文件格式是否为'.xlsx':", str(e))
            print("请重新上传文件。")
    return df

def download_mod(file_dLlink, save_dir, file_name, index):
    try:
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {file_dLlink} -d {save_dir} -o {file_name}
        print(f"第{index+1}行:{file_name}下载成功!")
    except Exception as e:
        print(f"第{index+1}行:{file_name}文件下载错误：", e)

from concurrent import futures

def start_mod():
    if mod_link:
        df = checkmod_link(mod_link)
    elif mod_path:
        df = checkmod_path(mod_path)
    else:
        df = checkmod_up()
    df = df.fillna("")

    with futures.ThreadPoolExecutor() as executor:
        future_to_index = {}

        for index, row in df.iterrows():
            file_switch, file_version, file_name, file_type, file_dLlink = row[["开关", "版本", "名称", "类型", "下载地址"]]

            if any(value == "" for value in [file_switch, file_version, file_name, file_type, file_dLlink]):
                print(f"跳过,第{index+1}行,因为:|{'|'.join([key for key, value in row.items() if value == ''])}|的内容不能为空!")
                continue

            save_dir = os.path.join(modelr_dir, file_version, file_type)
            file_path = os.path.join(save_dir, file_name)

            if not os.path.exists(file_path) and file_switch == "on":
                future = executor.submit(download_mod, file_dLlink, save_dir, file_name, index)
                future_to_index[future] = index

        for future in futures.as_completed(future_to_index):
            index = future_to_index[future]
            try:
                result = future.result()
            except Exception as e:
                print(f"第{index+1}行下载错误:", e)
    print("MOD运行完成...🎉")

if enable_mod:
  !apt-get -qq install -y aria2
  start_mod()

### (🔅)ControlNet

In [ ]:
# @title 💠ControlNet

# @markdown ##### 代码逻辑:不使用留空,需要使用选择与主模型对应的版本。(sdxl可以选择使用t2i_adapter)
# @markdown ##### 🎈可以在 "sd15_controlnets" 数组里删掉或者注释掉(#)不想下载的模型.
modelr_dir = f'{db_dir}/models'
_controlnet = "sd15" # @param ["", "sd15"]
sd15_controlnets = [
    "control_v11p_sd15_canny",
    "control_v11p_sd15_mlsd",
    "control_v11p_sd15_openpose",
    "control_v11p_sd15_inpaint",
    "control_v11e_sd15_ip2p",
    "control_v11e_sd15_shuffle",
    "control_v11p_sd15_softedge",
    "control_v11p_sd15_scribble",
    "control_v11p_sd15_lineart",
    "control_v11p_sd15_seg",
    "control_v11p_sd15_normalbae",
    "control_v11f1p_sd15_depth",
]

def download_controlnet(controlnet, controlnet_dir, split15_dir):
  try:
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {controlnet_dir} -o diffusion_pytorch_model.fp16.safetensors https://huggingface.co/lllyasviel/{controlnet}/resolve/main/diffusion_pytorch_model.fp16.safetensors
    #!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {controlnet_dir} -o diffusion_pytorch_model.safetensors https://huggingface.co/lllyasviel/{controlnet}/resolve/main/diffusion_pytorch_model.safetensors
    #!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {controlnet_dir} -o diffusion_pytorch_model.bin https://huggingface.co/lllyasviel/{controlnet}/resolve/main/diffusion_pytorch_model.bin
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {controlnet_dir} -o config.json https://huggingface.co/lllyasviel/{controlnet}/resolve/main/config.json
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {controlnet_dir} -o control_net_{split15_dir}.py https://huggingface.co/lllyasviel/{controlnet}/resolve/main/control_net_{split15_dir}.py
    #print(controlnet, ",下载成功!")
  except Exception as e:
    print(controlnet, "文件下载错误：", e)

if _controlnet == "sd15":
    sd15cn_dir = f"{modelr_dir}/sd-1/controlnet"

    with futures.ThreadPoolExecutor() as executor:
        future_to_controlnet = {}

        for controlnet in sd15_controlnets:
            split15_dir = controlnet.split("_sd15_")[1]
            controlnet_dir = f"{sd15cn_dir}/{split15_dir}"
            future = executor.submit(download_controlnet, controlnet, controlnet_dir, split15_dir)
            future_to_controlnet[future] = controlnet

        for future in futures.as_completed(future_to_controlnet):
            controlnet = future_to_controlnet[future]
            try:
                result = future.result()
                print(controlnet, "下载成功!")
            except Exception as e:
                print(controlnet, "文件下载错误：", e)
    print("ControlNet全部下载完成...🎉")

In [ ]:
# @title (可选)t2i_adapter
# @markdown #### 💠t2i_adapter类似与controlnet,用于图像控制.
# @markdown ##### 代码逻辑:不使用留空,需要使用选择与主模型对应的版本。(可重复运行)
# @markdown ##### 🎈可以在 "t2iadapters" 数组里删掉或者注释掉(#)不想下载的模型.
import concurrent.futures
modelr_dir = f'{db_dir}/models'
t2i_adapter = "" # @param ["", "sd15", "sdxl"]

sd15_t2iadapters = [
    "t2iadapter_canny_sd15v2",
    "t2iadapter_depth_sd15v2",
    "t2iadapter_sketch_sd15v2",
    "t2iadapter_zoedepth_sd15v1",
    "t2iadapter_openpose_sd14v1",
    "t2iadapter_keypose_sd14v1",
    "t2iadapter_seg_sd14v1",
    "t2iadapter_color_sd14v1",
]

sdxl_t2iadapters = [
    "t2i-adapter-sketch-sdxl-1.0",
    "t2i-adapter-depth-zoe-sdxl-1.0",
    "t2i-adapter-openpose-sdxl-1.0",
    "t2i-adapter-depth-midas-sdxl-1.0",
    "t2i-adapter-lineart-sdxl-1.0",
    "t2i-adapter-canny-sdxl-1.0",
]

if t2i_adapter == "sd15":
    sd15t2i_dir = f"{modelr_dir}/sd-1/t2i_adapter"

    def download_t2iadapter(t2iadapter):
        t2iadapter_dir = f"{sd15t2i_dir}/{t2iadapter}"
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {t2iadapter_dir} -o diffusion_pytorch_model.bin https://huggingface.co/TencentARC/{t2iadapter}/resolve/main/diffusion_pytorch_model.bin
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {t2iadapter_dir} -o config.json https://huggingface.co/TencentARC/{t2iadapter}/resolve/main/config.json
        print(t2iadapter,"下载成功!")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for t2iadapter in sd15_t2iadapters:
            futures.append(executor.submit(download_t2iadapter, t2iadapter))

        for future in concurrent.futures.as_completed(futures):
            future.result()

    print("t2i_adapter下载完成...🎉")

elif t2i_adapter == "sdxl":
    sdxlt2i_dir = f"{modelr_dir}/sdxl/t2i_adapter"

    def download_t2iadapter(t2iadapter):
        t2iadapter_dir = f"{sdxlt2i_dir}/{t2iadapter}"
        #!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {t2iadapter_dir} -o diffusion_pytorch_model.safetensors https://huggingface.co/TencentARC/{t2iadapter}/resolve/main/diffusion_pytorch_model.safetensors
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {t2iadapter_dir} -o diffusion_pytorch_model.fp16.safetensors https://huggingface.co/TencentARC/{t2iadapter}/resolve/main/diffusion_pytorch_model.fp16.safetensors
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {t2iadapter_dir} -o config.json https://huggingface.co/TencentARC/t2i-adapter-sketch-sdxl-1.0/resolve/main/config.json
        print(t2iadapter,"下载成功!")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for t2iadapter in sdxl_t2iadapters:
            futures.append(executor.submit(download_t2iadapter, t2iadapter))

        for future in concurrent.futures.as_completed(futures):
            future.result()

    print("t2i_adapter全部下载完成...🎉")

In [ ]:
# @title (可选)ip_adapter
# @markdown #### 💠ip_adapter类似与controlnet,用于图像控制.
ip_adapter = "" # @param ["", "sd15", "sdxl"]
# @markdown ##### 代码逻辑:不使用留空,需要使用选择与主模型对应的版本。(可重复运行)
# @markdown ##### 🎈可以在 "ipadapters" 数组里删掉不想下载的模型.
modelr_dir = f'{db_dir}/models'
if ip_adapter == "sd15":
    sd15ip_dir = f"{modelr_dir}/sd-1/ip_adapter"
    ipencoder_dir = f"{modelr_dir}/any/clip_vision/ip_adapter_sd_image_encoder"
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipencoder_dir} -o model.safetensors https://huggingface.co/InvokeAI/ip_adapter_sd_image_encoder/resolve/main/model.safetensors
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipencoder_dir} -o config.json https://huggingface.co/InvokeAI/ip_adapter_sd_image_encoder/resolve/main/config.json

    ipadapters = ["ip_adapter_sd15", "ip_adapter_sd15_light", "ip_adapter_plus_sd15", "ip_adapter_plus_face_sd15"]
    for ipadapter in ipadapters:
        ipadapter_dir = f"{sd15ip_dir}/{ipadapter}"
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipadapter_dir} -o ip_adapter.bin https://huggingface.co/InvokeAI/{ipadapter}/resolve/main/ip_adapter.bin
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipadapter_dir} -o image_encoder.txt https://huggingface.co/InvokeAI/{ipadapter}/resolve/main/image_encoder.txt
        print(ipadapter,"下载成功!")
elif ip_adapter == "sdxl":
    sdxlip_dir = f"{modelr_dir}/sdxl/ip_adapter"
    ipencoder_dir = f"{modelr_dir}/any/clip_vision/ip_adapter_sdxl_image_encoder"
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipencoder_dir} -o model.safetensors https://huggingface.co/InvokeAI/ip_adapter_sdxl_image_encoder/resolve/main/model.safetensors
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipencoder_dir} -o config.json https://huggingface.co/InvokeAI/ip_adapter_sdxl_image_encoder/resolve/main/config.json

    ipadapters = ["ip_adapter_sdxl", "ip_adapter_sdxl_vit_h", "ip_adapter_plus_sdxl_vit_h"]
    for ipadapter in ipadapters:
        ipadapter_dir = f"{sdxlip_dir}/{ipadapter}"
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipadapter_dir} -o ip_adapter.bin https://huggingface.co/InvokeAI/{ipadapter}/resolve/main/ip_adapter.bin
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {ipadapter_dir} -o image_encoder.txt https://huggingface.co/InvokeAI/{ipadapter}/resolve/main/image_encoder.txt
        print(ipadapter,"下载成功!")

### (⚒)附加功能：


In [ ]:
# @markdown #### 🌐使用云盘模型:
use_drivemodels = True # @param {type:"boolean"}
gmodelr_dir = "/content/drive/MyDrive/sd_models" # @param ["/content/drive/MyDrive/sd_models"] {allow-input: true}
# @markdown #### 📁打包输出图片:
use_zipoutputs = True # @param {type:"boolean"}
#@markdown #####保存间隔（秒）：
save_interval = 200 #@param {type:"integer"}
#@markdown #####图片输出目录：
outputs_path = "/content/IAekovnI/db/outputs" # @param ["/content/IAekovnI/db/outputs", "/content/drive/MyDrive/sd_outputs"] {allow-input: true}
#@markdown #####压缩保存目录：
zip_path = "/content/outputs" # @param ["/content/outputs", "/content/drive/sd_outputs"] {allow-input: true}

import os
def content_models():
  if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    print('Google Drive 挂载成功！')

  for modelver in ["sd-1", "sd-2", "sdxl", "sdxl-refiner"]:
    for subdir in ["main", "lora", "vae", "embedding"]:
      sub_dir_path = f"{gmodelr_dir}/{modelver}/{subdir}"
      os.makedirs(sub_dir_path, exist_ok=True)
      sub_dir_path = f"{gmodelr_dir}/{modelver}/{subdir}"

      if len(os.listdir(sub_dir_path)) == 0:
          continue
      os.system(f"ln -sf {sub_dir_path}/* {gmodelr_dir}/{modelver}/{subdir}/")

import shutil
import time
from datetime import datetime

def zip_outputs(outputs_path, zip_path, save_interval):
  save_count = 0
  last_backup_path = None

  while True:
    time.sleep(save_interval)

    save_count += 1
    backup_name = f"{datetime.now().strftime('%m_%d')}_outputs_{datetime.now().strftime('%H:%M')}_{save_count}"
    backup_path = f"{zip_path}/{backup_name}"

    if last_backup_path:
      os.remove(last_backup_path)

    try:
      shutil.make_archive(backup_path, 'zip', outputs_path)
      print(f'Backup created {backup_path}.zip')
      last_backup_path = f"{backup_path}.zip"
    except Exception as e:
      print(f"An error occurred while creating the backup: {e}")

import multiprocessing
def start_background_task(target_func, *args):
  p = multiprocessing.Process(target=target_func, args=args)
  p.start()
  return p

def stop_background_task(process):
  process.terminate()

if use_drivemodels:
  content_models()

# if use_zipoutputs:
#   outputs_paths = f"{outputs_path}/images"
#   back_zip = start_background_task(zip_outputs, outputs_paths, zip_path, save_interval)
#   # backup_thread = threading.Thread(target=zip_outputs, args=(outputs_paths, zip_path, save_interval))
#   # backup_thread.start()


### 🎨启动

In [11]:
# @title 内网穿透

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union
# @markdown #### [Ngrok](https://dashboard.ngrok.com/get-started/your-authtoken):
use_ngrok = True # @param {type:"boolean"}
ngrok_token = "" # @param {type:"string"}
# @markdown #### cloudflare:
use_cloudflare = True # @param {type:"boolean"}
# @markdown #### remote:
use_remote = True # @param {type:"boolean"}

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

if use_remote:
  id_rsa_file = f"{airoot_dir}/id_rsa"
  id_rsa_pub_file = f"{airoot_dir}/id_rsa.pub"
  if os.path.exists(id_rsa_file):
      os.remove(id_rsa_file)
  if os.path.exists(id_rsa_pub_file):
      os.remove(id_rsa_pub_file)
  ssh_name = "id_rsa"
  ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
  gen_key(ssh_path)

if use_ngrok:
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_token)

def start_tunnle():

  open("/content/output.log", "w").close()
  # 循环检测文件内容，直到存在"http://127.0.0.1:9090"。
  while not "http://127.0.0.1:9090" in open("/content/output.log", "r").read():
    time.sleep(1)

  #clear_output()
  from IPython.display import display, HTML
  audio_url = "https://github.com/Van-wise/sd-colab/raw/main/qidong.mp3"
  display(HTML(f'<audio src="{audio_url}" controls autoplay style="display:none"></audio>'))

  if use_ngrok:
      try:
          from pyngrok import ngrok
          ngrok_tunnel = ngrok.connect(9090, "http")
          print("ngrok_tunnel:", ngrok_tunnel)
      except Exception as e:
          print("ngrok 连接失败：", e)

  if use_cloudflare:
      try:
          from pycloudflared import try_cloudflare
          cloudflare_url = try_cloudflare(9090, verbose=False)
          print("cloudflare_tunnel:", cloudflare_url)
      except Exception as e:
          print("cloudflare 连接失败：", e)

  if use_remote:
      !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i {airoot_dir}/id_rsa remote.moe 2>&1 | tee -a /content/tunnel.log



# def tunnel():
#   open("/content/output.log", "w").close()

#   # 循环检测文件内容，直到存在"http://127.0.0.1:9090"。
#   while not "http://127.0.0.1:9090" in open("/content/output.log", "r").read():
#     time.sleep(1)

In [ ]:
# @title 启动
import threading
import sys
import time
# def tunnel():
#   open("/content/output.log", "w").close()

#   # 循环检测文件内容，直到存在"http://127.0.0.1:9090"。
#   while not "http://127.0.0.1:9090" in open("/content/output.log", "r").read():
#     time.sleep(1)

#   # 运行代码。
#   !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe 2>&1 | tee -a /content/tunnel.log

#tunnle = start_background_task(zip_outputs, outputs_paths, zip_path, save_interval)
if use_zipoutputs:
  outputs_paths = f"{outputs_path}/images"
  back_zip = start_background_task(zip_outputs, outputs_paths, zip_path, save_interval)
threading.Thread(target=start_tunnle, daemon=True).start()
!echo "3" | invokeai-web --root {main_dir}/db --ignore_missing_core_models --outdir $outputs_path 2>&1 | tee /content/output.log

### 关闭进程

In [ ]:
!pkill -f pyngrok
!pkill -f cloudflared
!pkill -f invokeai
stop_background_task(back_zip)
print("进程已关闭!")